In [1]:
from bigearthnet_encoder.encoder import tiff_dir_to_ben_s2_patch
from bigearthnet_common.example_data import get_s2_example_folder_path
from bigearthnet_common.base import get_s2_patch_directories

s2_path = get_s2_example_folder_path()
example_patch = [p for p in s2_path.iterdir()][0]

In [2]:
s2_patch_paths = get_s2_patch_directories(s2_path)
s2_patch_paths

[Path('/home/kaiclasen/.local/share/bigearthnet_common/BigEarthNet-S2-Example/S2A_MSIL2A_20170617T113321_4_55'),
 Path('/home/kaiclasen/.local/share/bigearthnet_common/BigEarthNet-S2-Example/S2A_MSIL2A_20170617T113321_36_85'),
 Path('/home/kaiclasen/.local/share/bigearthnet_common/BigEarthNet-S2-Example/S2A_MSIL2A_20171221T112501_56_35')]

In [4]:
from bigearthnet_encoder.encoder import write_S2_squirrel
from bigearthnet_encoder.squirrel_ext import ConfigurableMessagePackDriver

write_S2_squirrel(
    s2_path,
    "/tmp/delete_me",
)

1it [00:00,  7.83it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [5]:
msgpack_driver = ConfigurableMessagePackDriver("/tmp/delete_me/")
msgpack_driver

In [16]:
list(msgpack_driver.keys())
v = msgpack_driver.get_iter()
data = list(v)[0]


In [19]:
# data

In [20]:
import hashlib

# for alg in hashlib.algorithms_available:
#     try:
#         print(alg, hashlib.new(alg).digest_size)
#     except:
#         pass

s = "hello_world"
# hashlib.md5(s.encode(encoding="UTF-8")).hexdigest()
hashlib.md5(data)

TypeError: object supporting the buffer API required

In [23]:
80 / 23 * 50 / 60

2.898550724637681

In [3]:
hashlib.md5()
# s2_patch = tiff_dir_to_ben_s2_patch(example_patch)

In [4]:
# TODO: Compare performance impact between only storing the numpy arrays by their spatial resolution
# vs storing each band individually


In [5]:
import sys

# 160KB per patch with all bands
patch_size_in_bytes = sys.getsizeof(s2_patch.dumps())
patch_size_in_bytes

161607

In [6]:
2**30 // patch_size_in_bytes
# around 6600 patches per GB
# 80~

6644

In [7]:
import lmdb
from rich.progress import Progress
from bigearthnet_patch_interface.s2_interface import BigEarthNet_S2_Patch
import bigearthnet_common.constants as ben_constants
from typing import List

SHARD_SIZE = 6600
BATCH_SIZE = 16
REPS = 2
TOTAL_TEST_SIZE = SHARD_SIZE * REPS


def interpolate_to_ben_perf(size, seconds):
    interpolated_seconds = ben_constants.BEN_COMPLETE_SIZE / size * seconds
    return f"Would take {interpolated_seconds / 60:.02} min to pass through BigEarthNet"


def fake_lmdb_builder(fake_data, keys: List[str], lmdb_path: str = "S2_lmdb.db"):
    max_size = 2**40  # 1TebiByte
    env = lmdb.open(str(lmdb_path), map_size=max_size, readonly=False)

    with Progress() as progress:
        task = progress.add_task("Building LMDB archive", total=len(keys))
        for key in keys:
            with env.begin(write=True) as txn:
                txn.put(key.encode("utf-8"), fake_data)
                progress.update(task, advance=1)
        env.close()


keys = [f"{i:05}" for i in range(TOTAL_TEST_SIZE)]
# generate fake data and keys
# fake_lmdb_builder(s2_patch.dumps(), keys)

In [8]:
def read_fake_lmdb(keys, lmdb_path="S2_lmdb.db"):
    # readahead should be True if dataset fits in RAM
    # otherwise it may be faster to set readahead = False
    # as readonly=True no need for `locking` which _should_ take longer if lock=True
    env = lmdb.open(str(lmdb_path), readonly=True, readahead=True, lock=False)
    # possible optimization use single call to
    # getmulti(keys) instead of a new thread with a single element as transaction?

    for key in keys:
        with env.begin() as txn:
            byteflow = txn.get(key.encode("utf-8"))
            s2_patch = BigEarthNet_S2_Patch.loads(byteflow)


# ~4 sek to pass through 6600 * 2
# ~1.2sek to .loads
read_fake_lmdb(keys)

In [21]:
# interpolate_to_ben_perf(TOTAL_TEST_SIZE, 1.2)
interpolate_to_ben_perf(6600, 120)

'Would take 1.8e+02 min to pass through BigEarthNet'

In [10]:
import shutil
from pathlib import Path
from bigearthnet_encoder.squirrel_ext import (
    _patch_interface_to_dict,
)
from squirrel.iterstream import IterableSource
import numpy as np


In [11]:
import shutil
from pathlib import Path
from bigearthnet_encoder.squirrel_ext import (
    _patch_interface_to_dict,
    _write_s2_msgpack
    
)
from squirrel.iterstream import IterableSource


def gen_shards():
    while True:
        yield s2_patch_dict


it = IterableSource(iter(gen_shards()))
url = "dummy"
# Other supported compressions:
# fsspec.compression.available_compressions()
# [None, 'zip', 'bz2', 'gzip', 'lzma', 'xz']
compression = "None"
SAVE_URL = f"{url}_{compression}"
# p = Path(SAVE_URL)
# shutil.rmtree(p)
url = "../Ben_S2_None_Compression"
msgpack_driver = MyMessagePackDriver(url)

NameError: name 'MyMessagePackDriver' is not defined

In [20]:
from bigearthnet_common.constants import Split
from bigearthnet_common.base import get_original_split_from_patch_name
from bigearthnet_common.base import get_s2_patch_directories



1it [00:00,  9.22it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [ ]:
2.5 * B

In [8]:
# list(msgpack_driver.keys())
# 57s for normal threaded iterator
# >2min for ppool with 12 workers


In [9]:
from bigearthnet_common.sets import get_recommended_s2_patches, filter_patches_by_split, get_all_s2_patches

In [10]:
valids = get_recommended_s2_patches()

In [12]:
valids_train = filter_patches_by_split("S2", valids, "train")
s2_patches = get_all_s2_patches()
all_train = filter_patches_by_split("S2", s2_patches, "train")

In [13]:
# only async_map without anything else and calling b1_mean: 58s
# only async_map without anything else and lambda: 1m 4s
from concurrent.futures import ProcessPoolExecutor
from numba import njit

ppool = ProcessPoolExecutor(max_workers=8)

bands = "B01 B02 B03 B04 B05 B06 B07 B08 B8A B09 B11 B12".split(" ")

def b1(v):
    return v["B01"]

def mean(v):
    return v.mean()

def b1_mean(v):
    return mean(b1(v))

def val_train(v):
    return v["patch_name"] in valids_train

def in_all_train(v):
    return v["patch_name"] in all_train

def bands_maxes(v):
    return {b: v[b].max() for b in bands}

def bands_mines(v):
    return {b: v[b].min() for b in bands}

def bands_means(v):
    return {b: v[b].mean() for b in bands}

@njit(parallel=True)
def _numba_mean(a):
    return a.mean()


def numba_mean_b1(v):
    return _numba_mean(v["B01"])

# means = msgpack_driver.get_iter().filter(in_all_train).async_map(b1_mean, executor=ppool).collect()

# currently spawns 4178 threads... Requires 25% cpu utilization
# takes 1min25s
# means = msgpack_driver.get_iter().filter(in_all_train).async_map(numba_mean_b1).collect()

# with limiting the number of workers to 8 
# takes 1m19s
# produces "only" 1105 threads
# means = msgpack_driver.get_iter().filter(in_all_train).async_map(numba_mean_b1, max_workers=8).collect()

# Without numba call stack and 8 workers
# 88 threads and almost no CPU utilization
# 44s
# means = msgpack_driver.get_iter().filter(in_all_train).async_map(b1_mean, max_workers=8).collect()

# b1 mean with ProcessPool and 8 workers takes: 3m02s only train
# all_mins = msgpack_driver.get_iter().async_map(bands_means).collect()

In [14]:
# only train now 40s
# means = msgpack_driver.get_iter().filter(in_all_train).async_map(lambda v: v["patch_name"]).collect()

In [15]:
for i in msgpack_driver.get_iter():
    # print(i)
    break

In [17]:
from torch import Tensor
from torch.utils.data import DataLoader
from torch.nn.functional import interpolate
from squirrel.iterstream.torch_composables import TorchIterable
import numpy as np
from bigearthnet_common.constants import BEN_10m_CHANNELS, BEN_20m_CHANNELS
import torch

def interp_patches(patch_dict):
    bands_10m = np.stack([patch_dict[b] for b in BEN_10m_CHANNELS])
    bands_10m_torch = Tensor(np.float32(bands_10m)).unsqueeze(dim=0)

    bands_20m = np.stack([patch_dict[b] for b in BEN_20m_CHANNELS])
    bands_20m_torch = Tensor(np.float32(bands_20m)).unsqueeze(dim=0)

    bands_20m_interp = interpolate(
        bands_20m_torch,
        bands_10m.shape[-2:],
        mode="bicubic",
    )
    # print(bands_10m_torch.shape, bands_20m_interp.shape)
    tns = torch.concat((bands_10m_torch.squeeze(), bands_20m_interp.squeeze()), axis=0)
    # This is my magic normalization value to push it to the 0--1 range
    return tns / 10_000

In [29]:
import timm
import kornia.augmentation as K
from squirrel.iterstream.torch_composables import SplitByWorker
import pytorch_lightning as pl
from torch.optim import Adam

# WIP!
class FastModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model= timm.create_model("efficientnet_b0", pretrained=False, in_chans=10)
        self.criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=1e-3)
    
    def train_dataloader(self):
        it = (
            msgpack_driver.get_iter(key_hooks=[SplitByWorker]).filter(in_all_train).async_map(interp_patches, max_workers=16).compose(TorchIterable)
        )
        dl = DataLoader(it, batch_size=1024, pin_memory=True, num_workers=4)
        return dl
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self.model(inputs)
        loss = self.criterion(outputs, targets)
        return loss

augs = K.AugmentationSequential(
    K.RandomHorizontalFlip(),
    K.RandomVerticalFlip(),
    K.RandomResizedCrop(size=(120, 120)),
    # Setting this to true has a very limited performance impact
    # at least for a batch size of 1024
    same_on_batch=False,
)
# testing with 8 workers
# batch size 128
    # 10% CPU utilization with 656 threads
    # 2.7GB GPU memory 4% volatile memory
    # 4m30s for running through dataset on CUDA
    # dl = DataLoader(pipe, batch_size=128, pin_memory=True, num_workers=2)

    # with 2 workers:
    # CPU: 15% (not so sure...)
    # 2.7GB GPU memory but 21% volatile GPU-Util
    # only 1m31 sek :o

    # with 4 workers:
    # CPU: ~20%
    # 26% volatile GPU-util
    # only 1m10s for the entire pass with 4 workers...

# batch size 1024
# with 4 workers
    # CPU: 13%
    # VRAM: 6GB 
    # 1min

# batch size 10_000
# with 4 workers
    # CPU: <10%
    # VRAM: 10GB 
    # ~1m23s

# Testing with 16 max_workers for async_map patch interp
    # ~1m10s
    # smaller prefetch buffer had no effect
    # larger prefetch buffer increased RAM usage; nothing really changed the speed


# Applying augmentation on the CPU:
    # 30% CPU utilization
    # may spike up to 40%
    # takes 3m4s for CPU only with 10_024
    # could be different if real model is used with smaller batch size

# Nice! shuffle has almost no effect on the processing time!
# 1m10s
pipe = msgpack_driver.get_iter(key_hooks=[SplitByWorker]).filter(in_all_train).shuffle(10_000).async_map(interp_patches, max_workers=16).compose(TorchIterable)
# increasing number of workers for dataloader doesn't do a lot for my purposes

# storing data as list in memory is _not_ faster!
# reading from list with a single worker is actually slower!
# 1m26s
# even with setting the num_workers=2 the compute time is not significantly faster!
# it is even slower! Maybe due to the extra copying?
# >2m.15
# l = list(pipe)

dl = DataLoader(pipe, batch_size=1024, num_workers=2, pin_memory=True)
device = torch.device("cuda:0")
for _ in range(1):
    for b in dl:
        data = augs(b.to(device))

In [30]:
dl = DataLoader(pipe, batch_size=1024, pin_memory=True, num_workers=4)
device = torch.device("cuda:0")

# TODO: Actually test with a 'real' model!

for _ in range(1):
    for b in dl:
        data = augs(b.to(device))

In [1]:
import torch

torch.cuda.is_available()

/home/kaiclasen/.cache/pypoetry/virtualenvs/bigearthnet-encoder-sA-plAN6-py3.8/lib/python3.8/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 802: system not yet initialized (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [34]:
augs(b).shape

torch.Size([128, 10, 120, 120])

In [35]:
# async_map probably destroys the async_map torch.mean calculation
# 4m 35s for generating the 
means = msgpack_driver.get_iter().filter(in_all_train).async_map(interp_patches, max_workers=12).async_map(torch.mean).collect()

In [34]:
# for b in bands:
#     m = min(v[b] for v in all_mins)
#     print(b, " min: ", m)

In [21]:
# len(means) == len(all_train)

True

In [22]:
import numpy as np

# means of b1 mean: 711.450056265521
np.mean(means)

352.75488109716525

In [19]:
ben_constants.BAND_STATS_S2

{'mean': {'B01': 340.76769064,
  'B02': 429.9430203,
  'B03': 614.21682446,
  'B04': 590.23569706,
  'B05': 950.68368468,
  'B06': 1792.46290469,
  'B07': 2075.46795189,
  'B08': 2218.94553375,
  'B8A': 2266.46036911,
  'B09': 2246.0605464,
  'B11': 1594.42694882,
  'B12': 1009.32729131},
 'std': {'B01': 554.81258967,
  'B02': 572.41639287,
  'B03': 582.87945694,
  'B04': 675.88746967,
  'B05': 729.89827633,
  'B06': 1096.01480586,
  'B07': 1273.45393088,
  'B08': 1365.45589904,
  'B8A': 1356.13789355,
  'B09': 1302.3292881,
  'B11': 1079.19066363,
  'B12': 818.86747235}}

In [15]:
import bigearthnet_common.constants as ben_constants
assert len(names) == ben_constants.BEN_COMPLETE_SIZE

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from bigearthnet_encoder.squirrel_ext import _write_s2_msgpack
from bigearthnet_common.example_data import get_s2_example_folder_path
from bigearthnet_common.base import get_s2_patch_directories
from squirrel.iterstream import IterableSource
from pathlib import Path

# s2_path = get_s2_example_folder_path()
s2_path = Path("~/datasets/BigEarthNet-v1.0/BigEarthNet-v1.0").expanduser()
patch_paths = get_s2_patch_directories(s2_path)

In [ ]:
patch_paths = patch_paths[:100]

In [ ]:
_write_s2_msgpack(patch_paths, "dummy_None")

In [ ]:
list(msgpack_driver.keys())

In [ ]:
interpolate_to_ben_perf(6600, 60 * 2)

In [ ]:
# from functools import partial
# to_shard = partial(msgpack_driver.store.set, compression=None)
# batches = it.take(TOTAL_TEST_SIZE).batched(SHARD_SIZE, drop_last_if_not_full=False).map(to_shard).join()

In [ ]:
# GZIP: 6.5s for pass-through
# ~797MB
# 6.8
# with not prefetch_buffer 6.8s

# read using the messagepack driver

it_msg_pack = msgpack_driver.get_iter()
for item in it_msg_pack.take(10):
    # np.mean(item["10m_bands"])
    print(item["patch_name"])
    print(item["B01"])

In [ ]:
interpolate_to_ben_perf(TOTAL_TEST_SIZE, 0.7)